In [3]:
import os
import requests
import gpxpy

default_trace_route_url = "https://valhalla.mapzen.com/trace_route?api_key=mapzen-jLrDBSP&"
default_trace_attributes_url = "https://valhalla.mapzen.com/trace_attributes?api_key=mapzen-jLrDBSP&"

trace_route_url = os.getenv('valhalla_trace_route_url', default_trace_route_url)
trace_attributes_url = os.getenv('valhalla_trace_attributes_url', default_trace_route_url)

In [4]:
gpx_shape = []

with open("./Lien vers 2017-10-31_20-06-03.gpx", "r") as gpx_file:
    #parse gpx for mapzen
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coord = {}
                coord["lon"] = point.longitude
                coord["lat"] = point.latitude
                gpx_shape.append(coord)
                

In [5]:
#étape 1 : on passe dans trace_attributes             

shape_txt = str(gpx_shape)
shape_txt = shape_txt.replace("'","\"")
data = '{"shape":'+ shape_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 
ta1 = requests.post(trace_attributes_url,  data = data)

output_polyline = ta1.json()['shape']

In [6]:
#six degrees of precision in valhalla
inv = 1.0 / 1e6;

#decode an encoded string
def decode(encoded):
  decoded = []
  previous = [0,0]
  i = 0
  #for each byte
  while i < len(encoded):
    #for each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      #keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      #get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    #scale by the precision and chop off long coords also flip the positions so
    #its the far more standard lon,lat instead of lat,lon
    #decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    decoded.append({'lon': float('%.6f' % (ll[1] * inv)), 'lat': float('%.6f' % (ll[0] * inv))})
  #hand back the list of coordinates
  return decoded




In [7]:
#étape 2: on passe dans trace_route

shape_2_txt = str(decode(output_polyline))
shape_2_txt = shape_2_txt.replace("'","\"")
trace_route_data = '{"shape":'+ shape_2_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 

tr = requests.post(trace_route_url,  data = trace_route_data)
output_polyline2 = tr.json()['trip']['legs'][0]['shape']

#output_polyline2

In [8]:
#étape 3: on repasse dans trace_attribute

shape_3_txt = str(decode(output_polyline2))
shape_3_txt = shape_3_txt.replace("'","\"")
trace_attributes_data = '{"shape":'+ shape_3_txt +',"costing":"auto","directions_options":{"units":"miles"},"shape_match":"map_snap"}' 

ta = requests.post(trace_attributes_url,  data = trace_attributes_data)
#ta.json()

In [9]:
osm_way_list=[]
for a_way in ta.json()['edges']:
    osm_way_list.append('w' + str(a_way['way_id']))

#osm_way_list

josm_link = "http://localhost:8111/load_object?objects="
josm_link += ','.join(osm_way_list)
josm_link


'http://localhost:8111/load_object?objects=w188698755,w229874341,w166990132,w166990132,w166990132,w114314130,w24476110,w24516772,w229694404,w229694391,w24516772,w22973532,w22973532,w22973532,w22973532,w22973532,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w22973525,w266588351,w266588351,w266588351,w130009296,w40127559,w24520469,w24520469,w24520469,w24519822,w601138191,w60741307,w171562597,w171562599,w171562595,w24519688,w249018782,w249018790,w249018790,w249018790,w249018790,w249018784,w249018784,w249018784,w249018784,w614595043,w614595043,w614595043,w614595043,w614595043,w614595043,w259088530,w249018785,w614595043,w249018793,w264562897,w613159787,w259088549,w259088549,w188698743,w188698743,w188698743,w26114254,w26114254,w26071235,w26071235,w538570630,w538570630,w538570630,w22869590,w188698750,w139758897,w139758897,w22869581,w22869583,w22869583,w22869583,w22869583,w25739838,w25739838,w191550001,w191550001,w26055333,w26055333,w260553